# Sequential Feature Selection

In [ ]:
import pandas as pd
import numpy as np


df_bcf = pd.read_csv("https://raw.githubusercontent.com/edgarsmdn/MLCE_book/main/references/BCF_training.csv")

df_bcf.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score as R2

## Dataset Preparation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X, y = df_bcf.iloc[:, 3:].values, df_bcf.iloc[:, 2].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [47]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as MSE,r2_score as R2

model = KNeighborsRegressor(n_neighbors=5)

model.fit(X_train_std, y_train)

print('MSE_train:', MSE(y_train,model.predict(X_train)))
print('R2_train:', R2(y_train,model.predict(X_train)))
print('MSE_test:', MSE(y_test,model.predict(X_test)))
print('R2_test:', R2(y_test,model.predict(X_test)))

MSE_train: 2.2080624428749998
R2_train: -0.24141465919248084
MSE_test: 2.191513343
R2_test: -0.2546216866469089


## To understand how Sequential Feature selection works let's try Selecting the best 5 features for fixed k_neighbors using Scikit Learn

In [49]:
from sklearn.feature_selection import SequentialFeatureSelector as SFS
sfs_forward = SFS(model,
          n_features_to_select=6,
          direction='forward',
          scoring='neg_mean_squared_error',
          n_jobs=-1,
          cv=5)
sfs_forward = sfs_forward.fit(X_train_std, y_train)
X_train_std_transformed = sfs_forward.transform(X_train_std)
X_test_std_transformed = sfs_forward.transform(X_test_std)
model_transformed = KNeighborsRegressor(n_neighbors=5)
model_transformed.fit(X_train_std_transformed, y_train)
# print('MSE_test_transformed:', i, MSE(y_test,model_transformed.predict(X_test_std_transformed)))
print('R2_test_transformed:', R2(y_test,model_transformed.predict(X_test_std_transformed)))
selected_features = sfs_forward.get_support()

R2_test_transformed: 0.7866219221290096


## Rigorous features as well as k_neighbor search

In [ ]:
#TRY YOURSELF

#### **Gaussian Process using Numpy**

#### Defining kernel

In [ ]:
def kernel(x1,x2,l=1.0,sigma=1.0):
  d1 = (x1-x2.T)**2
  out = sigma*np.exp(-d1/(2+l**2))
  return out

#### Creating test set.

In [ ]:
np.random.seed(0)
n = 40
x_test = np.linspace(-4,4,n).reshape(-1,1)

#### Prior Sampling

It is important to know following derived expresions for kernels to implement Gaussian process


*   k = kernel(x_train,x_train); explains similarities between training set among themselves
*   k_star = kernel(x_train,x_test); explains similarities between train and test set
*   k_star_star = kernel(x_test,x_test); explains similarities between test set among themselves
*   k_inverse = np.linalg.inv(k)



1.   m_post = k_star.T.dot(k_inverse).dot(y_train)          # mathematically defined predictions (mean) (see lecture slides)
2.   cov_post = k_star_star - k_star.T.dot(k_inverse).dot(k_star)   # mathematically defined covariance matrix (see lecture slides)
3.   std = np.sqrt(np.diag(cov_post)).reshape(m_post.shape)        # variance is always diagnol elements of covariance matrix




In [ ]:
m = np.zeros(x_test.shape)
cov = kernel(x_test,x_test)   # k_star_star
prior_predictions = np.random.multivariate_normal(m.reshape(-1),cov,4)     #completely random without even knowing training data points)
print(prior_predictions.shape,x_test.shape)

#### Plotting Prior Samples/Predictions

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_test,prior_predictions.T)
plt.xlim(-4,4)
plt.ylim(-3,3)
plt.show()

#### Define training data

In [ ]:
x_train = np.array([-3,-2,-1,1,2]).reshape(-1,1)
y_train = np.cos(x_train)
noise = 1e-07

#### Calculation of Posteriors

In [ ]:
k = kernel(x_train,x_train)      # explains similarities between training set among themselves
k_star = kernel(x_train,x_test)  # explains similarities between train and test set
k_star_star = kernel(x_test,x_test)   # explains similarities between test set among themselves
k_inverse = np.linalg.inv(k)

#### Posteriors mean and covariance matrix



In [ ]:
m_post = k_star.T.dot(k_inverse).dot(y_train)          # mathematically defined predictions (mean) (see lecture slides)
cov_post = k_star_star - k_star.T.dot(k_inverse).dot(k_star)   # mathematically defined covariance matrix (see lecture slides)
std = np.sqrt(np.diag(cov_post)).reshape(m_post.shape)        # variance is always diagnol elements of covariance matrix

#### Draw posterior samples/predictions



In [ ]:
posterior_predictions = np.random.multivariate_normal(m_post.ravel(),cov_post,4)

#### Plotting Posterior Samples/Predictions

In [ ]:
plt.plot(x_train,y_train,'o',ms=8)
plt.plot(x_test,posterior_predictions.T,'--')
plt.gca().fill_between(np.squeeze(x_test),np.squeeze(m_post - 1.96*std),np.squeeze(m_post + 1.96*std), color='aliceblue')
plt.plot(x_test,m_post)
plt.xlim(-4,4)
plt.ylim(-3,3)
plt.show()

In [ ]:
plt.gca().fill_between(np.squeeze(x_test),np.squeeze(m_post - 1.96*std),np.squeeze(m_post + 1.96*std), color='aliceblue')
plt.plot(x_test,m_post)
plt.xlim(-4,4)
plt.ylim(-3,3)
plt.show()

#### **Gaussian Process using Scikit Learn**

In [ ]:
from sklearn.gaussian_process import kernels, GaussianProcessRegressor
np.random.seed(0)
n = 40
kernel_ = [kernels.RBF(),kernels.RationalQuadratic(),kernels.DotProduct(sigma_0=1.0)**2,kernels.RationalQuadratic()*kernels.Matern(),kernels.Matern()]

In [ ]:
for kernel in kernel_:
  # implementation of Gaussian process
  gp = GaussianProcessRegressor(kernel = kernel)

  # defining test set
  x_test = np.linspace(-4,4,n).reshape(-1,1)

  # prior sample/predictions
  m_prior, std_prior = gp.predict(x_test, return_std = True)
  prior_predictions = gp.sample_y(x_test,2)

  print('#'*50)
  print(kernel)
  print('#'*50)

  # plotting the prior_predictions/samples (initilaization)
  plt.figure(figsize = (10,3))
  plt.subplot(1,2,1)
  plt.plot(x_test,m_prior)
  plt.fill_between(x_test.ravel(),m_prior -std_prior, m_prior + std_prior, color = 'aliceblue')
  plt.plot(x_test,prior_predictions, '--')
  plt.title('prior')

  # defining training dataset
  x_train = np.array([-3,-2,-1,1,2]).reshape(-1,1)
  y_train = np.cos(x_train)
  gp.fit(x_train,y_train)

  # posterior predictions
  m_post, std_post = gp.predict(x_test,return_std=True)
  m_post = m_post.reshape(-1)
  post_predictions = np.squeeze(gp.sample_y(x_test,3))

  # plotting the post_predictions/samples (initilaization)
  plt.subplot(1,2,2)
  plt.plot(x_test,m_post)
  plt.scatter(x_train,y_train,color='blue',s=50)
  plt.fill_between(x_test.ravel(),m_post -std_post, m_post + std_post, color = 'aliceblue')
  plt.plot(x_test,post_predictions, '--')
  plt.title('posterior')

  print("gp.kernel_ : " , gp.kernel_)
  print("gp.log_marginal_likelihood : " , gp.log_marginal_likelihood(gp.kernel_.theta))

#### Let's try Linear Regression to approximate cosine function as we did above using Gaussian Process and see the difference

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
np.random.seed(0)
n = 40
x_train = np.array([-3,-2,-1,1,2]).reshape(-1,1)
x_test = np.linspace(-4,4,n).reshape(-1,1)
y_train = np.cos(x_train)

In [ ]:
pf = PolynomialFeatures(degree=3)
pf.fit(x_train)
x_train_pf = pf.transform(x_train)
x_test_pf = pf.transform(x_test)
model = LinearRegression()
model.fit(x_train_pf,y_train)
y_pred_test = model.predict(x_test_pf)

In [ ]:
plt.figure(figsize = (10,3))
plt.scatter(x_train,y_train,color='blue',s=50)
plt.plot(x_test,y_pred_test, '--')
plt.xlim(-4,4)
plt.ylim(-5,2)

#### As we can see Linear regression has good intrapolation  but very poor extrapolation while GP can extrapolate well too if we use good kernels.

#### Let's try Gaussian Process on BCF dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X, y = df_bcf.iloc[:, 3:].values, df_bcf.iloc[:, 2].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# sc = StandardScaler()
# X_train_std = sc.fit_transform(X_train)
# X_test_std = sc.transform(X_test)

In [ ]:
kernel_ = [kernels.RationalQuadratic(alpha=0.25,length_scale=0.25)]
# kernel_ = [kernels.RBF(),kernels.RationalQuadratic(),kernels.DotProduct(sigma_0=1.0)**2,kernels.RationalQuadratic()*kernels.Matern(),kernels.Matern()]


In [ ]:
for kernel in kernel_:
  # implementation of Gaussian process
  gp = GaussianProcessRegressor(kernel = kernel)

  # defining test set
  x_test = X_test_std

  # prior sample/predictions
  m_prior, std_prior = gp.predict(x_test, return_std = True)
  prior_predictions = gp.sample_y(x_test,2)

  print('#'*50)
  print(kernel)
  print('#'*50)


  # defining training dataset
  x_train = X_train_std
  gp.fit(x_train,y_train)

  # posterior predictions
  m_post, std_post = gp.predict(x_test,return_std=True)
  m_post = m_post.reshape(-1)
  post_predictions = np.squeeze(gp.sample_y(x_test,3))
  R2_score = R2(y_test, m_post)
  print( R2_score)
  plt.figure(figsize = (8,4))
  plt.scatter(y_test,m_post)